In [28]:
#import python library 
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import mysql.connector
import re
import cv2
from PIL import Image
import requests
import numpy as np
import math

In [7]:
#connect to MysQl Data Base
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="scraping_data"
)
mycursor = mydb.cursor()

In [8]:
#get rate of numbers between to strings
def get_rate(product1, product2):    
    regex = r'[0-9]+'
    numbers1 = set(re.findall(regex, product1))
    numbers2 = set(re.findall(regex, product2))
    union = numbers1.union(numbers2)
    intersection = numbers1.intersection(numbers2)
    if len(numbers1)==0 and len(numbers2) == 0:
        rate = 0
    else:
        rate = (len(intersection)/ len(union))
    return int(rate*100)

In [9]:
# load Mysql table supplier_product data with filter by supplier id
def loadData(supid):
    list=[]
    query="SELECT * FROM supplier_products where supplier_id = %s"
    val=(supid,)
    try:
        mycursor.execute(query, val)
        results = mycursor.fetchall()
        for res in results:
            dict={
                'id':res[0],
                'name':res[1],
                'catgory':res[10],
                'des':res[5],
                'brand_id':res[9],
                'img':res[13]
            }
            list.append(dict)
    except:
         list=[]
    return list

In [ ]:
def loadMatch(id):
    list1=[]
    list2=[]
    list=[]
    query="SELECT * FROM matching where prod_id_one = %s"
    val=(id,)
    mycursor.execute(query, val)
    results = mycursor.fetchall()
    for res in results:
        dict1={
            'id1':res[1],
            'id2':res[2],
            'score':res[8]
        }
        list1.append(dict1)
    query="SELECT * FROM matching where prod_id_two = %s"
    val=(id,)
    mycursor.execute(query, val)
    results = mycursor.fetchall()
    for res in results:
        dict2={
            'id1':res[2],
            'id2':res[1],
            'score':res[8]
        }
        list2.append(dict1)     
    list=list1+list2
    return list

In [ ]:
def in_group_exist_verify(id1,id2):
    list1=loadMatch(id1)
    list2=loadMatch(id2)
    for l1 in list1:
        for l2 in list2:
            if(l1['id'] == l2['id'] > 85):
                return False
    return True

In [10]:
#load mysql table supplier_category with id filter
def getCatgory(uid):
    query="SELECT * FROM supplier_categories where id = %s"
    val=(uid,)
    try:
        mycursor.execute(query, val)
        result = mycursor.fetchone()[1]
    except:
         result=None
    return str(result)

In [11]:
#load mysql table supplier_brands with id filter
def getBrand(uid):
    query="SELECT * FROM supplier_brands where id = %s"
    val=(uid,)
    try:
        mycursor.execute(query, val)
        result = mycursor.fetchone()[1]
    except:
         result=None
    return str(result)

In [12]:
# return true when the percentage of  similarty between two strings is higher than 75%
def similar(ch1,ch2):
    result = fuzz.partial_ratio(ch1.lower(), ch2.lower())
    if result > 75:
        return True
    else:
        return False

In [23]:
# load mysql data from supplier_product_attributes and concate them as strings
def getAttr(id):
    string=''
    query="SELECT * FROM supplier_product_attributes where supplier_product_id=%s"
    val=(id,)
    mycursor.execute(query, val)
    results = mycursor.fetchall()
    #print(len(results))
    for res in results:
        string=string + res[1]+' '+res[2]+' '
    return string

In [24]:
#calcul the similarity  betwwen tow attributes and return the result
def AttributeScore(prod1,prod2):
    ch1=getAttr(prod1)
    ch2=getAttr(prod2)
    result = fuzz.partial_ratio(ch1.lower(), ch2.lower())
    return result

In [29]:
#calcul the degree of resemblance between two images
def imageSimilarty(url, url1):
    # image1
    try:
        img = Image.open(requests.get(url, stream=True).raw)
        gray_image = cv2.cvtColor(np.float32(img), cv2.COLOR_BGR2GRAY)
        histogram = cv2.calcHist([gray_image], [0], 
                                 None, [256], [0, 256])
    except:
        return 1
    # image2
    try: 
        img1 = Image.open(requests.get(url1, stream=True).raw)
        gray_image1 = cv2.cvtColor(np.float32(img1), cv2.COLOR_BGR2GRAY)
        histogram1 = cv2.calcHist([gray_image1], [0], 
                                  None, [256], [0, 256])
    except:
        return 1
    c1, c2 = 0, 0

    # Euclidean Distace between data1 and test
    i = 0
    while i<len(histogram) and i<len(histogram1):
        c1+=(histogram[i]-histogram1[i])**2
        i+= 1
    c1 = c1**(1 / 2)
    c=c1[0]
    return math.floor(100-(c/1000))

In [30]:
#save matching data into data base
def saveMatching(id1,id2,nameScore,nameNumberRate,desScore,brandScore,imgScore,attrScore,score):
    if(score > 60):
        query="INSERT INTO matching (prod_id_one, prod_id_two, name_score, name_number_rate, des_score, brand_score, image_score, attr_score, score) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        val=(id1, id2, nameScore, nameNumberRate, desScore, brandScore, imgScore, attrScore, score)
        mycursor.execute(query, val)
        mydb.commit()

In [ ]:
# main algorithm matching process
list1 = loadData(16)
for x in range(17,26):
    list2= loadData(x)
    for l1 in list1:
        print("-------------->",l1['id'])
        for l2 in list2:
            if(similar(getCatgory(l1['catgory']), getCatgory(l2['catgory']))):
                nameScore= fuzz.partial_ratio(l1['name'].lower(), l2['name'].lower())
                desScore= fuzz.partial_ratio(str(l1['des']).lower(), str(l2['des']).lower())
                nameNumberRate=get_rate(str(l1['name']),str(l2['name']))
                brandScore=fuzz.partial_ratio(str(getBrand(l1['brand_id'])).lower(), str(getBrand(l2['brand_id'])).lower())
                imgScore = imageSimilarty(str(l1['img']), str(l2['img']))
                attrScore=AttributeScore(str(l1['id']),str(l2['id']))
                moy=((nameScore*10) + (nameNumberRate*10) +(desScore*2) + (brandScore*10) + (imgScore*2) + (attrScore*3))//37
                print(f'{nameScore}-->{nameNumberRate}-->{desScore}-->{brandScore}-->{imgScore}-->{attrScore}-->{moy}')
                saveMatching(l1['id'],l2['id'],nameScore,nameNumberRate,desScore,brandScore,imgScore,attrScore,moy)

--------------> 35
50-->14-->71-->25-->83-->49-->36
55-->28-->71-->100-->88-->50-->62
55-->28-->71-->100-->88-->50-->62
60-->28-->71-->100-->87-->50-->63
52-->12-->57-->100-->85-->50-->56
55-->28-->71-->100-->88-->50-->62
55-->12-->57-->100-->88-->50-->57
55-->12-->57-->100-->88-->50-->57
58-->12-->57-->100-->87-->50-->57
50-->14-->86-->25-->79-->50-->37
53-->12-->86-->100-->85-->50-->57
52-->12-->86-->100-->85-->50-->57
55-->28-->71-->100-->86-->50-->62
49-->12-->71-->0-->90-->50-->29
55-->12-->71-->100-->88-->50-->57
48-->0-->71-->25-->79-->50-->31
52-->0-->71-->100-->85-->50-->53
52-->0-->71-->100-->85-->50-->53
55-->12-->71-->100-->86-->50-->57
47-->0-->71-->0-->90-->50-->25
51-->0-->71-->25-->79-->50-->32
55-->12-->71-->100-->88-->50-->57
49-->0-->80-->25-->79-->50-->32
48-->0-->80-->25-->79-->50-->32
53-->11-->80-->25-->82-->52-->37
45-->10-->80-->0-->69-->52-->27
61-->0-->80-->100-->88-->55-->57
53-->10-->80-->100-->89-->53-->57
45-->0-->80-->0-->69-->52-->24
52-->0-->80-->25-->

47-->11-->80-->25-->90-->52-->35
64-->0-->80-->100-->76-->53-->57
44-->0-->80-->0-->86-->53-->25
44-->0-->80-->0-->86-->53-->25
45-->0-->80-->0-->60-->53-->24
48-->0-->80-->25-->89-->52-->33
45-->0-->80-->25-->90-->52-->32
46-->0-->80-->0-->61-->52-->24
48-->0-->80-->0-->60-->54-->24
67-->0-->71-->100-->76-->55-->57
46-->0-->71-->0-->86-->55-->25
47-->0-->71-->0-->86-->55-->25
63-->0-->71-->100-->80-->57-->56
47-->0-->57-->25-->89-->55-->31
60-->0-->57-->100-->80-->57-->55
48-->0-->57-->25-->90-->55-->32
47-->0-->57-->25-->89-->55-->31
47-->0-->57-->25-->89-->55-->31
45-->0-->57-->25-->89-->55-->31
46-->0-->57-->0-->60-->55-->23
45-->0-->57-->0-->86-->55-->24
45-->0-->57-->0-->86-->55-->24
65-->0-->71-->100-->76-->55-->57
47-->0-->57-->25-->90-->55-->31
45-->0-->71-->25-->90-->55-->32
47-->11-->57-->0-->91-->56-->28
48-->0-->57-->0-->91-->56-->25
56-->11-->71-->100-->88-->55-->58
55-->11-->71-->100-->88-->55-->57
52-->10-->57-->100-->85-->55-->55
56-->0-->57-->100-->88-->55-->54
55-->0

52-->0-->57-->0-->63-->50-->24
50-->0-->57-->0-->68-->52-->24
51-->0-->57-->0-->59-->52-->24
49-->0-->57-->0-->63-->51-->23
52-->12-->57-->0-->73-->52-->28
43-->0-->57-->0-->65-->50-->22
44-->0-->71-->0-->60-->50-->23
54-->0-->57-->0-->73-->52-->25
55-->0-->57-->0-->73-->52-->26
48-->0-->57-->0-->46-->50-->22
53-->0-->57-->25-->62-->49-->31
46-->0-->57-->25-->67-->49-->29
49-->0-->57-->100-->66-->50-->50
46-->11-->57-->0-->57-->50-->25
46-->0-->57-->100-->59-->50-->49
50-->0-->57-->100-->66-->50-->51
48-->0-->57-->0-->57-->50-->23
50-->12-->57-->0-->70-->50-->27
48-->0-->57-->0-->73-->52-->24
46-->0-->57-->25-->67-->49-->29
54-->0-->57-->33-->59-->52-->34
53-->0-->57-->25-->62-->49-->31
47-->0-->57-->0-->46-->50-->22
52-->0-->57-->0-->70-->50-->24
49-->0-->57-->0-->70-->52-->24
49-->0-->57-->0-->70-->52-->24
44-->0-->57-->25-->65-->52-->29
53-->0-->57-->0-->68-->52-->25
44-->0-->57-->25-->65-->52-->29
48-->0-->57-->100-->66-->50-->50
50-->0-->71-->25-->71-->50-->32
53-->0-->57-->0-->70

44-->0-->71-->0-->66-->52-->23
45-->0-->71-->25-->65-->52-->30
39-->0-->71-->0-->55-->52-->21
44-->0-->71-->0-->67-->50-->23
47-->0-->57-->0-->45-->50-->22
51-->0-->57-->25-->70-->50-->31
44-->0-->57-->0-->77-->54-->23
53-->0-->57-->0-->68-->52-->25
46-->0-->57-->0-->45-->50-->22
44-->0-->57-->25-->65-->52-->29
39-->0-->57-->0-->55-->52-->20
48-->0-->57-->0-->70-->52-->24
52-->0-->57-->0-->68-->52-->25
43-->0-->57-->25-->65-->52-->29
51-->0-->57-->0-->62-->50-->24
56-->0-->57-->33-->59-->52-->34
47-->0-->57-->0-->44-->49-->22
49-->0-->57-->0-->70-->52-->24
46-->0-->57-->0-->45-->50-->22
54-->0-->71-->33-->59-->52-->34
54-->0-->71-->33-->61-->52-->34
54-->0-->71-->33-->59-->52-->34
54-->0-->57-->33-->61-->52-->34
45-->0-->57-->0-->66-->52-->23
56-->0-->71-->100-->66-->50-->53
54-->0-->86-->0-->71-->52-->27
48-->12-->71-->0-->69-->50-->27
48-->0-->71-->0-->70-->52-->24
39-->0-->71-->0-->55-->52-->21
44-->0-->71-->25-->65-->52-->30
48-->0-->71-->100-->64-->50-->51
52-->0-->57-->0-->68-->5

47-->0-->57-->0-->61-->50-->23
54-->0-->71-->0-->89-->52-->27
48-->0-->71-->0-->87-->49-->25
50-->0-->57-->0-->88-->52-->25
51-->0-->57-->100-->82-->50-->52
54-->0-->57-->0-->89-->52-->26
46-->0-->57-->0-->84-->52-->24
46-->0-->57-->0-->84-->52-->24
53-->0-->57-->0-->90-->64-->27
40-->0-->57-->0-->65-->52-->21
41-->0-->57-->0-->74-->52-->22
45-->0-->57-->0-->84-->52-->24
44-->0-->57-->25-->83-->52-->30
50-->0-->57-->0-->77-->50-->24
46-->0-->57-->0-->86-->50-->24
41-->0-->57-->0-->82-->63-->23
43-->0-->57-->25-->83-->52-->30
53-->0-->71-->0-->89-->52-->27
40-->0-->71-->0-->82-->54-->23
53-->0-->57-->0-->89-->52-->26
47-->0-->57-->0-->84-->52-->24
51-->0-->57-->100-->82-->50-->52
56-->0-->57-->100-->75-->50-->53
40-->0-->57-->0-->65-->52-->21
48-->0-->57-->0-->60-->49-->23
50-->0-->57-->0-->88-->52-->25
54-->0-->57-->0-->89-->52-->26
47-->0-->57-->0-->84-->52-->24
44-->0-->57-->25-->83-->52-->30
43-->0-->57-->25-->83-->52-->30
50-->0-->71-->25-->72-->50-->32
53-->0-->57-->0-->89-->52-->

...

66-->28-->86-->100-->79-->50-->65
61-->12-->86-->0-->83-->49-->32
69-->12-->71-->100-->86-->50-->61
66-->0-->71-->25-->92-->50-->37
55-->0-->71-->100-->89-->49-->54
56-->0-->71-->100-->87-->49-->54
65-->12-->71-->100-->79-->50-->60
60-->0-->71-->0-->83-->49-->28
67-->0-->71-->25-->92-->50-->37
69-->12-->71-->100-->86-->50-->61
67-->0-->80-->25-->92-->50-->38
66-->0-->80-->25-->92-->50-->37
47-->11-->80-->25-->92-->51-->35
45-->10-->80-->0-->64-->51-->26
59-->0-->80-->100-->84-->50-->55
56-->10-->80-->100-->81-->51-->57
45-->0-->80-->0-->64-->51-->24
45-->0-->80-->25-->92-->51-->32
47-->11-->80-->25-->92-->51-->35
67-->11-->80-->100-->80-->51-->60
48-->10-->80-->0-->90-->51-->29
53-->10-->80-->0-->63-->51-->28
47-->10-->80-->0-->90-->51-->28
45-->0-->80-->0-->64-->51-->24
47-->0-->80-->25-->92-->51-->32
45-->0-->80-->25-->92-->51-->32
58-->0-->80-->100-->84-->50-->55
48-->11-->80-->25-->93-->51-->36
64-->0-->80-->100-->80-->51-->57
46-->0-->80-->0-->90-->51-->25
45-->0-->80-->0-->90-->5

45-->11-->57-->25-->78-->49-->33
64-->11-->57-->100-->72-->49-->58
47-->11-->57-->25-->79-->49-->33
47-->11-->57-->25-->78-->49-->33
45-->11-->57-->25-->78-->49-->33
45-->11-->71-->25-->78-->49-->33
48-->10-->57-->0-->52-->49-->25
46-->10-->57-->0-->76-->49-->26
47-->10-->57-->0-->76-->49-->26
63-->11-->71-->100-->67-->49-->58
45-->11-->57-->25-->79-->49-->33
45-->11-->57-->25-->79-->49-->33
47-->25-->71-->0-->80-->48-->31
48-->11-->57-->0-->80-->48-->27
58-->25-->57-->100-->76-->49-->60
59-->25-->57-->100-->76-->49-->60
62-->22-->71-->100-->75-->49-->61
56-->11-->57-->100-->76-->49-->56
58-->11-->57-->100-->76-->49-->56
47-->11-->57-->0-->80-->48-->26
59-->20-->71-->100-->74-->53-->60
58-->11-->57-->100-->76-->49-->56
59-->11-->57-->100-->76-->49-->57
47-->0-->71-->0-->54-->49-->23
48-->0-->57-->17-->82-->62-->30
49-->10-->57-->0-->76-->49-->27
51-->0-->71-->0-->69-->49-->25
42-->0-->57-->0-->75-->49-->22
49-->0-->57-->0-->76-->49-->24
52-->0-->57-->0-->78-->49-->25
50-->0-->57-->0-->

46-->0-->57-->0-->51-->51-->22
49-->0-->57-->0-->65-->51-->23
54-->0-->57-->0-->71-->51-->25
46-->0-->57-->0-->72-->51-->23
46-->10-->71-->0-->62-->51-->26
49-->0-->57-->0-->64-->51-->23
47-->0-->57-->0-->72-->51-->23
46-->0-->57-->0-->62-->51-->23
52-->11-->57-->25-->74-->51-->35
48-->0-->57-->25-->77-->54-->31
54-->0-->57-->0-->71-->51-->25
52-->0-->57-->0-->64-->54-->24
49-->0-->57-->0-->65-->51-->23
47-->0-->57-->0-->51-->51-->22
52-->0-->57-->25-->74-->51-->32
47-->0-->57-->25-->74-->54-->30
47-->0-->57-->25-->74-->54-->30
44-->0-->57-->17-->70-->54-->27
51-->0-->57-->25-->72-->54-->31
44-->0-->57-->17-->70-->54-->27
47-->0-->57-->0-->72-->51-->23
52-->0-->57-->0-->76-->51-->25
53-->0-->57-->25-->74-->51-->32
46-->0-->57-->0-->67-->51-->23
47-->0-->57-->0-->62-->51-->23
42-->0-->57-->0-->54-->54-->21
48-->0-->57-->0-->50-->51-->22
57-->0-->57-->100-->73-->56-->54
47-->0-->57-->0-->62-->51-->23
55-->0-->71-->0-->66-->54-->26
55-->0-->71-->0-->64-->54-->26
53-->0-->57-->25-->74-->51

44-->0-->71-->25-->68-->56-->30
51-->0-->71-->25-->59-->53-->31
47-->0-->57-->0-->38-->52-->22
44-->0-->71-->17-->57-->53-->27
59-->0-->71-->100-->47-->54-->53
48-->0-->71-->25-->61-->53-->31
52-->0-->71-->25-->59-->53-->32
43-->0-->71-->17-->57-->53-->27
47-->0-->71-->0-->53-->52-->23
50-->0-->71-->0-->51-->54-->24
49-->0-->71-->0-->38-->52-->23
47-->0-->71-->25-->61-->53-->30
47-->0-->57-->0-->38-->52-->22
50-->0-->57-->0-->51-->54-->23
50-->0-->57-->0-->52-->54-->23
50-->0-->57-->0-->51-->54-->23
50-->0-->57-->0-->52-->54-->23
45-->0-->57-->0-->57-->54-->22
52-->0-->57-->0-->58-->52-->24
52-->0-->57-->25-->62-->53-->31
48-->11-->71-->0-->60-->52-->27
50-->0-->57-->25-->61-->53-->30
58-->0-->57-->100-->47-->54-->52
44-->0-->57-->17-->57-->53-->26
48-->0-->57-->0-->56-->52-->23
54-->0-->57-->25-->59-->53-->31
50-->0-->57-->25-->61-->53-->30
45-->0-->57-->0-->57-->54-->22
59-->0-->71-->100-->55-->62-->54
43-->0-->57-->17-->57-->53-->26
58-->0-->57-->0-->62-->52-->26
50-->0-->71-->0-->5

50-->0-->71-->25-->62-->53-->31
42-->0-->71-->0-->58-->51-->22
50-->0-->57-->25-->63-->53-->31
42-->0-->57-->0-->59-->52-->21
42-->0-->57-->0-->59-->52-->21
53-->0-->57-->25-->64-->61-->32
44-->0-->57-->0-->45-->53-->21
54-->7-->57-->100-->51-->53-->53
40-->0-->57-->0-->59-->53-->21
46-->0-->57-->17-->59-->53-->27
51-->0-->57-->0-->53-->51-->23
53-->0-->57-->100-->61-->51-->51
53-->8-->57-->100-->58-->62-->54
46-->0-->57-->17-->59-->53-->27
47-->0-->57-->25-->63-->53-->30
53-->0-->57-->0-->58-->55-->25
49-->0-->57-->25-->63-->53-->30
42-->0-->57-->0-->59-->52-->21
40-->0-->57-->0-->58-->51-->21
49-->0-->57-->0-->51-->51-->23
44-->0-->57-->0-->45-->53-->21
41-->0-->57-->0-->41-->51-->20
48-->0-->57-->25-->62-->53-->30
50-->0-->57-->25-->63-->53-->31
42-->0-->57-->0-->59-->52-->21
46-->0-->57-->17-->59-->53-->27
46-->0-->57-->17-->59-->53-->27
52-->0-->57-->0-->49-->51-->23
47-->0-->57-->25-->63-->53-->30
53-->8-->71-->100-->58-->62-->55
40-->0-->57-->0-->59-->52-->21
48-->0-->57-->0-->5